In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


In [ ]:
#CONDITIONS
import re
import requests
from bs4 import BeautifulSoup
!pip install cloudscraper
import cloudscraper
import string
from tqdm import tqdm
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')



# dummy_words = ['that','as', 'an','and','on','in','for','the','or','of','to','its','a','other','this','is','about','it','can','these','may','cause','with','also','called','because','order','when','why','who','at','alone','such','does','with','without','are','your','into', 'make', 'making', 'from', 'so','are','time','exercise','diet',            "too","much","between","one","two","weight","treated","alcohol","days","if","over","way", ]





     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
def getAllConditions():
  """
  webcraping function to get paragraphs for many conditions from medlineplus
  """
  locs = ["https://medlineplus.gov/foodandnutrition.html", "https://medlineplus.gov/bloodheartandcirculation.html", "https://medlineplus.gov/bonesjointsandmuscles.html", "https://medlineplus.gov/brainandnerves.html", "https://medlineplus.gov/digestivesystem.html", "https://medlineplus.gov/earnoseandthroat.html", "https://medlineplus.gov/endocrinesystem.html", "https://medlineplus.gov/eyesandvision.html", "https://medlineplus.gov/immunesystem.html", "https://medlineplus.gov/kidneysandurinarysystem.html", "https://medlineplus.gov/lungsandbreathing.html", "https://medlineplus.gov/mouthandteeth.html", "https://medlineplus.gov/femalereproductivesystem.html", "https://medlineplus.gov/skinhairandnails.html", "https://medlineplus.gov/malereproductivesystem.html"]
  all_conds = []
  for l in locs:
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(l).text
    soup = BeautifulSoup(my_text, "html.parser")
    matches = soup.find(id = 'tpgp')
    links = [a.get('href') for a in matches.find_all('a', href=True)]
    link2 = []
    cter = 0;
    for l2 in links:
      if '.html' in l2:
        link2.append(l2)
        cter = cter + 1
    all_conds = all_conds + link2

  all_unique_conds = np.unique(np.array(all_conds))

  cond_desc = []
  for cond in all_unique_conds:
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(cond).text
    soup = BeautifulSoup(my_text, "html.parser")
    matches_2 = soup.find_all(id = "topsum_section")
    matches_summary = matches_2[0]
    desc = matches_summary.getText()
    desc = desc.lower()
    desc = desc.replace('\n', ' ')
    cond_desc.append(desc)
  return np.array(cond_desc)

paragraphs_conds = getAllConditions()
print(paragraphs_conds.shape)

(575,)


In [ ]:
def getAllMeds():
  """
  webcraping function to get paragraphs for many medications from medlineplus
  """
  locs = []
  for s in string.ascii_uppercase:
    locs.append('https://medlineplus.gov/druginfo/drug_' + s + 'a.html')
  
  all_meds = []
  for l in locs:
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(l).text
    soup = BeautifulSoup(my_text, "html.parser")
    matches = soup.find(id = 'index')
    links = [a.get('href') for a in matches.find_all('a', href=True)]
    link2 = []
    for l2 in links:
      if '.html' in l2:
        link2.append(l2)
    all_meds = all_meds + link2

  all_unique_meds = np.unique(np.array(all_meds))
  print(all_unique_meds.shape)

  med_desc = []
  for med in all_unique_meds:
    try:
      myUrl = 'https://medlineplus.gov/druginfo' + med[1:]
      scraper = cloudscraper.create_scraper(delay = 15)  
      my_text = scraper.get(myUrl).text
      soup = BeautifulSoup(my_text, "html.parser")
      matches_2 = soup.find_all(id = "why")
      desc = matches_2[0].contents[1].text
      desc = desc.lower()
      desc = desc.replace('\n', ' ')
      med_desc.append(desc)
    except:
      print("error")
      continue
  return np.array(med_desc)

paragraphs_meds = getAllMeds()

(1604,)
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error


In [ ]:
def getAllRandom(numRands = 5000, max_words_per_page = 150):
  """
  webcraping function to get max_words_per_page words from numRands articles 
  from wikipedia
  """
  locs = []
  for s in range(numRands):
    locs.append('https://en.wikipedia.org/wiki/Special:Random')
  print(f"fetched all {numRands} articles")

  rand_desc = []
  for l in tqdm(locs):
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(l).text
    soup = BeautifulSoup(my_text, "html.parser")
    desc = soup.find(class_='mw-parser-output')
    descs = desc.find_all('p')
    desc = ""
    for d in descs:
      desc = desc + d.getText()

    desc = ' '.join(desc.split()[:max_words_per_page])
    desc = desc.lower()
    desc = desc.replace('\n', ' ')
    rand_desc.append(desc)

  return rand_desc

In [ ]:
def get_corpus(paragraphs):
  """
  Input: list of strings
  Takes each string and strips it so that only nouns remain 
  Output: tuple, first item is a list of noun-only strings, 
    second is list of integers representing the length of the noun-only strings
  """
  nltk_types = ['NN', 'NNP', 'NNS', 'NNPS']  #nltk.help.upenn_tagset() to see docum.
  corpus = []
  lens = []
  for par in paragraphs:
    tokens = nltk.word_tokenize(par)
    tagged = nltk.pos_tag(tokens)
    nouns_only = ' '.join([word for word, typ in tagged if typ in nltk_types])
    corpus.append(nouns_only)
    lens.append(len(nouns_only.split()))
  lens = np.array(lens)[:, None]
  return corpus, lens


In [ ]:
corpus_conds, cond_lens = get_corpus(paragraphs_conds)
corpus_meds, med_lens = get_corpus(paragraphs_meds)
paragraphs_rands = getAllRandom()
corpus_rands, rand_lens = get_corpus(paragraphs_rands)
corpus_conds_rand = corpus_conds + corpus_rands
corpus_meds_rand = corpus_meds + corpus_rands
corpus_full = corpus_conds + corpus_meds + corpus_rands
vectorizer = CountVectorizer(ngram_range= (1, 1))
cfn = vectorizer.fit_transform(corpus_full)


  0%|          | 0/5000 [00:00<?, ?it/s]

getting random corpus
fetched all 5000 articles


100%|██████████| 5000/5000 [45:07<00:00,  1.85it/s]


In [ ]:
import pickle
import os.path
from os import path
from google.colab import drive

get_random_corpus = True
include_random_corpus = True

drive.mount('/content/drive')
if path.exists('/content/drive/MyDrive/corpus.pickle'):
  with open('/content/drive/MyDrive/corpus.pickle', 'rb') as handle:
    corpus_conds, cond_lens, corpus_meds, med_lens, corpus_rands, rand_lens, corpus_conds_rand, corpus_meds_rand, vectorizer = pickle.load(handle)

else:
  with open('/content/drive/MyDrive/corpus.pickle', 'wb') as handle:
    pickle.dump([corpus_conds, cond_lens, corpus_meds, med_lens, corpus_rands, rand_lens, corpus_conds_rand, corpus_meds_rand, vectorizer], handle, protocol=pickle.HIGHEST_PROTOCOL)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
